# Have a first look at the trajectory data

This will load the trajectory data just as it is generated by [Parcels](https://oceanparcels.org).

## Technical preamble

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

import intake  # data catalogs

import xarray as xr  # provides basic datatypes for labeled nd arrays

In [ ]:
import dask
from dask.distributed import Client, wait

# Make sure the Dask dashboard is easy to reach
dask.config.set(
    {
        'distributed.dashboard.link':
        "{JUPYTERHUB_BASE_URL}user/{JUPYTERHUB_USER}/{JUPYTERHUB_SERVER_NAME}/proxy/{port}/status"
    }
)

# start a Dask cluster that spans a whole node
client = Client(n_workers=1, threads_per_worker=64, memory_limit=240e9)
client

## Set parameters

Let's make sure that this is the only place where users have to change contents.

In [ ]:
# parameters
platform = "jsc_scratch"

## Open the data catalog

In [ ]:
catalog = intake.open_catalog(f"../intake-catalogs/medseaconnectivity_{platform}.yaml")
print(list(catalog))

## Load a dataset

There's two sets of trajectories:
- `'medsea-trajectories-stokes'` contains trajectories at the sea surface that are subject to drift by waves and the Ocean circulation.
- `'medsea-trajectories'` contains trajectories at the sea surface and in deeper layers that are only subject to the Ocean circulation.

For now, we'll only use the smaller `'medsea-trajectories-stokes'` dataset. Let's load it:

In [ ]:
dataset = catalog['medsea-trajectories-stokes'].to_dask()
display(dataset)
print(f"amounts to {dataset.nbytes / 1e9} GB")

## Details about the data

Dimensions
- `"traj"`: Trajectory ID
- `"obs"`: Time step (not indicative of the point in time this positions was sampled but just enumerating the time step since the trajectory started)

Data variables:
- `"time"`: Time stamps for each position.
- `"z"`: Vertical component (counted below the sea surface) for each position
- `"lat"`: [Latitude](https://en.wikipedia.org/wiki/Geographic_coordinate_system#Latitude_and_longitude) for each position
- `"lon"`: [Longitude](https://en.wikipedia.org/wiki/Geographic_coordinate_system#Latitude_and_longitude) for each position
- `"distance"`: along-trajectory distance traveled since trajectory started
- `"temp"`: Temerature felt at each position
- `"land"`: Influcence of land-points felt at each position. See below for details.
- `"MPA"`: Marine Protected Area this trajectory started in

Why sample `"land"` influence? — Due to post-processing of the simulated Ocean current fields (originally meant to make it more convenient to work with the data...), the velocity data used to drive these virtual fish larvae are not accurately respecting the (horizontal) continuity equation. In the open Ocean this is not much of a problem as the error introduced is of the same order as the discretization error (~grid resolution of the velocity data). Close to the coast, however, the error can lead to spurious beaching or unrealistically slow ocean currents very close to the coast. To be able to handle larvae that come too close to the coast in post processing, we've detected the fraction of land data that influenced the larva at each position.

What about the `"MPA"`'s? — There are many [mostly near-shore protected areas in the Mediterranean Sea](https://drive.google.com/file/d/0Bw8D-TFFFccxNUwyVXBONHdjUVk/view) that are meant to ensure stable population of marine animals. The data we use here stems from a study that investigated the genetic connectivity of these MPSs with each other and with other non-protected parts of the Mediterranean Sea by letting virtual fish larvae spawn in different MPAs and then travel passively with the Ocean currents.

What can we learn from `"temp"`? — The simulated larvae have a comfort temperature which allows them to live and thrive as they drift with the currents. If the temperature is too high or too low, this can have negative effects on the larvae.

## Have a first look at the data

We'll use [HoloViews](http://holoviews.org/) and other tools from the [Pyviz](https://pyviz.org/) and [HoloViz](https://holoviz.org/) universe to look at a subset of the positions in the dataset.

In [ ]:
import holoviews as hv
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.operation import decimate
hv.extension('bokeh')

In [ ]:
%%time

# create 1D arrays of all positions (not discriminating between different trajectories etc.)
lon = dataset.lon.data.flatten()
lat = dataset.lat.data.flatten()

# note that as everything is Dask arrays execution is immediate

In [ ]:
%%time

points = hv.Points((lon, lat))

# still very fast execution as the real calculation will only be done as we request numbers

In [ ]:
# this creates a plot. Watch the Dask dashboard as the plot is generated.
datashade(points).opts(width=800, height=600)